# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [1]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [5]:
query= "SELECT * FROM crime_scene_report WHERE type = 'murder' AND city = 'SQL City' AND date = '20180115'"
pd.set_option('display.max_colwidth',None)
ejecutar_query(query=query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [9]:
query ='SELECT * FROM PERSON WHERE address_street_name = "Northwestern Dr"'

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
1,12711,Norman Apolito,667757,599,Northwestern Dr,778264744
2,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
3,15171,Weldon Penso,336999,311,Northwestern Dr,131379495
4,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
5,18376,Josh Shi,653712,1091,Northwestern Dr,193899001
6,19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
7,22239,Dusty Sigafus,710517,1125,Northwestern Dr,724386723
8,23044,Val Portlock,924989,3143,Northwestern Dr,100593316
9,23960,Kristopher Lagerberg,658777,1392,Northwestern Dr,492912529


In [12]:
query = 'SELECT MAX(address_number) FROM PERSON WHERE address_street_name = "Northwestern Dr"'

ejecutar_query(query=query)

,MAX(address_number)
0,4919


In [13]:
query ='SELECT name FROM PERSON WHERE address_number = 4919'
ejecutar_query(query=query)

,name
0,Morty Schapiro


In [14]:
query = 'SELECT id FROM PERSON WHERE name = "Morty Schapiro"'

ejecutar_query(query=query)

,id
0,14887


In [16]:
query ='SELECT transcript FROM INTERVIEW WHERE person_id = 14887'

ejecutar_query(query=query)

,transcript
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [29]:
query= 'SELECT id FROM GET_FIT_NOW_MEMBER WHERE membership_status = "gold"'

ejecutar_query(query=query)

,id
0,NL318
1,0YJ24
2,5Y28Y
3,A5N3S
4,R4J4S
...,...
63,0T622
64,BL3KC
65,48Z7A
66,48Z55


In [31]:
query = 'SELECT name FROM GET_FIT_NOW_MEMBER WHERE id LIKE "48Z%"'
ejecutar_query(query=query)

,name
0,Tomas Baisley
1,Joe Germuska
2,Jeremy Bowers


In [25]:
query= 'SELECT check_in_date FROM GET_FIT_NOW_CHECK_IN WHERE membership_id = "48Z%"'

ejecutar_query(query=query)

,check_in_date


In [33]:
query= 'SELECT id FROM DRIVERS_LICENSE WHERE plate_number LIKE "%H42W%"'

ejecutar_query(query=query)

,id
0,183779
1,423327
2,664760


In [36]:
query= 'SELECT name FROM PERSON WHERE license_id = "183779" OR license_id = "423327" OR license_id = "664760"'

ejecutar_query(query=query)

,name
0,Tushar Chandra
1,Jeremy Bowers
2,Maxine Whitely


In [38]:
query= 'SELECT id FROM PERSON WHERE name = "Jeremy Bowers"'

ejecutar_query(query=query)

,id
0,67318


In [39]:
query = 'SELECT transcript FROM INTERVIEW WHERE person_id = "67318"'

ejecutar_query(query=query)

,transcript
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [40]:
query = 'SELECT name FROM PERSON WHERE address_street_name = "Franklin Ave"'
ejecutar_query(query=query)

,name
0,Wilmer Wolever
1,Annabel Miller
2,Johnnie Schee
3,Carleen Etoll
4,Zachary Ybarbo
5,Gema Nantz
6,Clarita Rickels
7,Shelby Dezeeuw
8,Amado Mattan
9,Cordell Lindamood


In [41]:
query = 'SELECT id FROM PERSON WHERE name = "Annabel Miller"'

ejecutar_query(query=query)

,id
0,16371


In [43]:
query = 'SELECT transcript FROM INTERVIEW WHERE person_id = "16371"'

ejecutar_query(query=query)

,transcript
0,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


# EL ASESINO ES JEREMY BOWERS

### SU INTERROGATORIO

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n

### VAMOS A POR EL VILLANO!!

In [44]:
query = 'SELECT * FROM DRIVERS_LICENSE'

pd.set_option('display.max_colwidth',10)

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS
...,...,...,...,...,...,...,...,...,...
10002,999923,19,77,amber,black,female,5L0ZI4,GMC,Sierra...
10003,999940,71,61,green,green,male,1B8QN8,Mitsub...,Eclipse
10004,999981,67,69,brown,blue,female,1684K3,Land R...,LR2
10005,999986,49,58,green,grey,male,F8F64H,Lexus,LS


In [47]:
query = 'SELECT id FROM DRIVERS_LICENSE WHERE hair_color = "red" AND height = "67" OR height = "65" AND car_make = "Tesla"'

ejecutar_query(query=query)

,id
0,121750
1,147044
2,173984
3,181622
4,197377
5,260364
6,270494
7,315095
8,360548
9,379396


In [48]:
query = 'SELECT car_model FROM DRIVERS_LICENSE WHERE car_make = "Tesla"'

ejecutar_query(query=query)


,car_model
0,Roadster
1,Model S
2,Model S
3,Model 3
4,Model S
5,Roadster
6,Roadster
7,Roadster
8,Model S
9,Model S


In [50]:
query = 'SELECT id FROM DRIVERS_LICENSE WHERE hair_color = "red" AND car_model = "Model S"'

ejecutar_query(query=query)

,id
0,202298
1,291182
2,736081
3,918773


In [52]:
query = 'SELECT * FROM FACEBOOK_EVENT_CHECKIN'

ejecutar_query(query=query)

,person_id,event_id,event_name,date
0,28508,5880,Nudist...,20170913
1,63713,3865,but th...,20171009
2,63713,3999,...,20170502
3,63713,6436,Old pr...,20170926
4,82998,4470,Help a...,20171022
...,...,...,...,...
20006,99716,1143,SQL Sy...,20171206
20007,99716,1143,SQL Sy...,20171212
20008,99716,1143,SQL Sy...,20171229
20009,67318,4719,The Fu...,20180115


In [54]:
query = 'SELECT id FROM PERSON WHERE license_id ="202298" OR license_id = "291182" OR license_id = "736081" OR license_id = "918773"'

ejecutar_query(query=query)

,id
0,57410
1,78881
2,90700
3,99716


In [55]:
query = 'SELECT event_name FROM FACEBOOK_EVENT_CHECKIN WHERE person_id = "57410" OR person_id = "78881" OR person_id = "90700" OR person_id ="99716"'

ejecutar_query(query=query)

,event_name
0,harder.\n
1,God di...
2,knows ...
3,SQL Sy...
4,SQL Sy...
5,SQL Sy...


In [57]:
query = 'SELECT person_id FROM FACEBOOK_EVENT_CHECKIN WHERE event_name = "SQL Symphony Concert" AND date LIKE "201712%"'

ejecutar_query(query=query)

,person_id
0,62596
1,19260
2,58898
3,69699
4,19292
5,43366
6,92343
7,28582
8,28582
9,81526


In [64]:
query = 'SELECT * FROM PERSON WHERE id = "99716"'

pd.set_option('display.max_colwidth',None)

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


# LA VILLANA ES MIRANDA PRIESTLY